In [1]:
import pandas as pd
import os
import sys

In [2]:
# Set paths
homey = os.getcwd()
redouxPath = os.path.join(homey, 'ForecastRedoux')
sqlPath = os.path.join(redouxPath, 'SQL')
rawDataPath = os.path.join(redouxPath, 'RawData')
AdditionalInfoPath = os.path.join(homey, 'AdditionalInfo')

In [5]:
# Pull in most recent timeline
timeline = pd.read_excel(os.path.join(homey, 'RegularForecast.xlsx'), sheetname='Timeline')

In [6]:
# Connect to FB API
sys.path.insert(0, 'Z:\Python projects\FishbowlAPITestProject')
import connecttest

In [7]:
# Query part costs
myresults = connecttest.create_connection(sqlPath, 'AvgCostQuery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'AvgCosts.xlsx')

Success!


In [8]:
# Get part costs into a dataFrame
costpath = os.path.join(rawDataPath, 'AvgCosts.xlsx')
avgCost = pd.read_excel(costpath) #Opens and puts the data into a dataframe
avgCost = avgCost.sort_values(by='PART', ascending=True) #Sort the data by part

In [9]:
# Merge part costs onto timeline
timeline = pd.merge(timeline.copy(), avgCost.copy(), how='left', on='PART')

In [10]:
# Calculate total costs
timeline['TotalCost'] = timeline['QTYREMAINING'] * timeline['AvgCost']

In [11]:
# Grab list of imaginary builds and create build number reference
newbuilds = pd.read_excel(os.path.join(AdditionalInfoPath, 'PartsToBuild.xlsx'), sheetname='Sheet1')
newbuilds.reset_index(inplace=True)
newbuilds['buildIndex'] = (newbuilds['index'] * -1) -1
newbuilds.drop('index', axis=1, inplace=True)

In [12]:
# Create columns to be filled by the next loop.
newbuilds['Total_Purchase'] = 0
newbuilds['Total_Cost_Produced'] = 0

In [13]:
""" This loop calculates and updates the total purchasing need for each imaginary build
    and it also finds the total raw material cost. """
for index, row in newbuilds.iterrows():
    imagBuild = row['buildIndex']
    thisBuild = timeline[timeline['GRANDPARENT'] == imagBuild].copy()
    purchDF = thisBuild[thisBuild['ORDERTYPE'] == 'Purchase'].copy()
    totPurchDF = purchDF[purchDF['TotalCost'].notnull()].copy()
    newbuilds.set_value(index, 'Total_Purchase', purchDF['TotalCost'].sum())
    # Now adding the total raw good cost of "Buy" items to show total amount in movement
    buyItemsDF = thisBuild[thisBuild['Make/Buy'] == 'Buy'].copy()
    rawGoodsDF = buyItemsDF[buyItemsDF['ORDERTYPE'] == 'Raw Good']
    newbuilds.set_value(index, 'Total_Cost_Produced', rawGoodsDF['TotalCost'].sum())
    # to be saved for lack of costing info, currently not used!!!
    noCostDF = rawGoodsDF[rawGoodsDF['TotalCost'].isnull()].copy() # ... it's so nice ...
    # what a waste, eh?

In [15]:
newbuilds

,Part,Qty,Date,buildIndex,Total_Purchase,Total_Cost_Produced
0,016-1632-10 r01,38,2017-05-15,-1,16413,-30283
1,016-452-10 r03,54,2017-05-15,-2,2772,-17474
2,016-1622-10 r02,35,2017-05-15,-3,837,-12269
3,016-1922-10 r02,40,2017-05-15,-4,170889,-195876
4,016-1009-10 r02,40,2017-05-15,-5,23351,-57566
5,016-131-10,50,2017-05-15,-6,22113,-24885
6,016-1877-10 r03,370,2017-05-15,-7,182903,-196660
7,016-143-10,30,2017-05-15,-8,10377,-17536
8,016-138-10,20,2017-05-15,-9,3795,-9437
9,016-1847-10 r01,10,2017-05-15,-10,4446,-6386


In [16]:
newbuilds.to_clipboard()

In [17]:
noCostDF

,ORDER,ITEM,ORDERTYPE,PART,QTYREMAINING,INV,DATESCHEDULED,PARENT,Make/Buy,GRANDPARENT,DESCRIPTION,AvgCost,TotalCost
18176,-1031,Imaginary,Raw Good,960-165-10 R02,-250.0,0.0,2017-05-14,-16,Buy,-16,Box; Crdbrd; 24 X 15-3/8 X 13-3/8; White Ou...,NaN,NaN
18206,-1031,Imaginary,Raw Good,960-167-10 R02,-250.0,0.0,2017-05-14,-16,Buy,-16,Foam; White; 24 X 15-1/4 X 6; top/bottom fo...,NaN,NaN
